In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import socket

# TCP_IP = "192.168.50.126" # 应该不同电脑上这里的IP是一样的，换成实验场景中连HoloLens的电脑的IP，但是下面的端口需要换成对应的。
# TCP_IP = "172.28.131.16" # 应该不同电脑上这里的IP是一样的，换成实验场景中连HoloLens的电脑的IP，但是下面的端口需要换成对应的。
TCP_IP = "127.0.0.1" # 应该不同电脑上这里的IP是一样的，换成实验场景中连HoloLens的电脑的IP，但是下面的端口需要换成对应的。
# TCP_PORT = 11000
TCP_PORT = 12000
MESSAGE = b"Hello, Unity 2!"
# sock = socket.socket()
# sock.connect((TCP_IP, TCP_PORT))
mp_drawing = mp.solutions.drawing_utils
mp_selfie_segmentation = mp.solutions.selfie_segmentation

# For webcam input:
# BG_COLOR = (0, 0, 0) # black
BG_COLOR = (0, 0, 0, 0) # black
idx = 0
cap = cv2.VideoCapture(0)
with mp_selfie_segmentation.SelfieSegmentation(
    model_selection=1) as selfie_segmentation:
  bg_image = None
  while cap.isOpened():
    idx = idx + 1
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    
    # Here maybe I will need some human tracking for segmenting 360 input
    
    
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = selfie_segmentation.process(image)

    image.flags.writeable = True
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGRA)

    # Draw selfie segmentation on the background image.
    # To improve segmentation around boundaries, consider applying a joint
    # bilateral filter to "results.segmentation_mask" with "image".
    condition = np.stack(
      # (results.segmentation_mask,) * 3, axis=-1) > 0.1
      (results.segmentation_mask,) * 4, axis=-1) > 0.1
    # The background can be customized.
    #   a) Load an image (with the same width and height of the input image) to
    #      be the background, e.g., bg_image = cv2.imread('/path/to/image/file')
    #   b) Blur the input image by applying image filtering, e.g.,
    #      bg_image = cv2.GaussianBlur(image,(55,55),0)
    if bg_image is None:
      bg_image = np.zeros(image.shape, dtype=np.uint8)
      bg_image[:] = BG_COLOR
    output_image = np.where(condition, image, bg_image)
    # output_image_black = np.where(condition, image, bg_image)

    # Make the background transparent
    # output_image = cv2.cvtColor(output_image_black, cv2.COLOR_BGR2BGRA)
    # for i in range(0,output_image_black.shape[0]): #访问所有行
    #     for j in range(0,output_image_black.shape[1]): #访问所有列
    #         if output_image_black[i,j,0] == 0 and output_image_black[i,j,1] == 0 and output_image_black[i,j,2] == 0:
    #             output_image[i,j,3] = 0

    # The way to output the result.
    cv2.imshow('MediaPipe Selfie Segmentation', output_image)
    # encode_param=[int(cv2.IMWRITE_JPEG_QUALITY),40]
    encode_param=[int(cv2.IMWRITE_PNG_COMPRESSION), 0]
    # result, imgencode = cv2.imencode('.jpg', output_image, encode_param)
    result, imgencode = cv2.imencode('.png', output_image, encode_param)
    output_image_data = np.array(imgencode)
    b = output_image_data.tobytes()

    sock = socket.socket()
    sock.connect((TCP_IP, TCP_PORT))
    sock.send(b)
    sock.close()
#     b = bytearray(f)
#     cv2.imwrite(r'D:\Projects\Playground\Jupyter\MediaPipe\tmp\selfie_segmentation_output' + str(idx) + '.png', output_image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
sock.close()
cv2.destroyAllWindows()

In [3]:
pip install mediapipe --user

  Using cached mediapipe-0.10.21-cp311-cp311-win_amd64.whl.metadata (10 kB)
  Using cached jax-0.6.1-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.6.1-cp311-cp311-win_amd64.whl.metadata (1.2 kB)
  Using cached matplotlib-3.10.3-cp311-cp311-win_amd64.whl.metadata (11 kB)
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.5.2-py3-none-win_amd64.whl.metadata (1.6 kB)
  Using cached cffi-1.17.1-cp311-cp311-win_amd64.whl.metadata (1.6 kB)
  Using cached ml_dtypes-0.5.1-cp311-cp311-win_amd64.whl.metadata (22 kB)
  Using cached contourpy-1.3.2-cp311-cp311-win_amd64.whl.metadata (5.5 kB)
Using cached mediapipe-0.10.21-cp311-cp311-win_amd64.whl (51.0 MB)
Using cached sounddevice-0.5.2-py3-none-win_amd64.whl (363 kB)
Using cached cffi-1.17.1-cp311-cp311-win_amd64.whl (181 kB)
Using cached jax-0.6.1-py3-none-any.whl (2.4 MB)
Using cached jaxlib-0.6.1-cp311-cp311-win_amd64.whl (56.8 MB)
Using cached ml_dtypes-0.5.1-cp311